written by kevin lee



In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import pytz
from datetime import datetime, timezone
import os
import sys

from __future__ import absolute_import



%cd "/content/drive/My Drive/TraCAR/darknet2"
cur_dir = %pwd
sys.path.append(cur_dir)

%cd "/content/drive/My Drive/TraCAR/toolkit-master/got10k"
cur_dir = %pwd
sys.path.append(cur_dir)

%cd "/content/drive/My Drive/TraCAR/toolkit-master/got10k/trackers"
cur_dir = %pwd
sys.path.append(cur_dir)

%cd "/content/drive/My Drive/TraCAR/toolkit-master/got10k/utils"
cur_dir = %pwd
sys.path.append(cur_dir)

%cd "/content/drive/My Drive/TraCAR/siamrpn-pytorch"
cur_dir = %pwd
sys.path.append(cur_dir)

from viz import show_frame

from trackers import Tracker
from siamrpn import TrackerSiamRPN
from mosse import mosse


# Arguments

In [0]:
siamrpn_path = "/content/drive/My Drive/TraCAR/siamrpn-pytorch/pretrained/siamrpn/model.pth"
args_darknet_path = "/content/drive/My Drive/TraCAR/darknet2"

args_confidence = 0.5
args_threshold = 0.3

eval_train = True
eval_test = False
eval_val = True

args_dataset_path = "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR"
gt_dir = os.path.join(args_dataset_path, "labels")

train_path = os.path.join(args_dataset_path, "training")
test_path = os.path.join(args_dataset_path, "testing")
val_path = os.path.join(args_dataset_path, "validation")

args_vid_path = "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR/testing/track0139"
args_img_path = "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR/testing/track0150/track0150[01].png"

In [0]:
labelsPath = os.path.sep.join([args_darknet_path, "data", "UFPR-ALPR.names"])
LABELS = open(labelsPath).read().strip().split("\n")
np.random.seed(100)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# Init Model

In [0]:
# derive the paths to the YOLO weights and model configuration
# weightsPath = os.path.sep.join([yolo_path, "yolov3.weights"])
weightsPath = os.path.sep.join([args_darknet_path, "KLbackup","UFPR","v12","KLyolov2-tiny-UFPR_12000.weights"])
configPath = os.path.sep.join([args_darknet_path, "KLbackup","UFPR","v12","Copy of KLyolov2-tiny-UFPR.cfg"])
# load our YOLO object detector trained on UFPR-ALPR
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# Helper Functions


## Utils

In [0]:
def filter_and_sort(files_list):
  # look through video folder, and filter out txt files. then sort pngs by frame number
  filtered_list = []
  frame_nums = []
  for i in range(len(files_list)):
    if files_list[i][-3] != 'p' or files_list[i][-1].isdigit() or files_list[i][0] != 't' or '(' in files_list[i]: 
      continue
    else: # is png file
      frame_nums.append(int(files_list[i][-7:-5])) 
      filtered_list.append(files_list[i])

  sorted_indices = sorted(range(len(frame_nums)), key=lambda k: frame_nums[k])

  output_list = [filtered_list[sorted_indices[j]] for j in range(len(sorted_indices))]
  return output_list

def center2corner(box, H=1080, W=1920):
  # converts bbox from normalized yolo [x_c, y_c, w, h] to non-normalized [x1, y1, x2, y2]
  box[0] = int( (box[0] - box[2]/2)*W )
  if box[0] < 0:
    box[0] = 0
  if box[0] > W:
    box[0] = W

  box[1] = int( (box[1] - box[1]/2)*H )
  if box[0] < 0:
    box[0] = 0
  if box[0] > H:
    box[0] = H

  box[2] = (box[0]+box[2]*W)
  box[3] = (box[1]+box[3]*H)

  return box


def IOU(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def eval_image(det_info, gt_det_info, iou_threshold=0.5):
  # only consider the detection with the highest confidence
  tp = 0
  fp = 0
  fn = 0
  if len(det_info) == 0:
    fn += 1
  else:
    # if det_info[0][0] != gt_det_info[0]: # class ids dont match
    #   fp += 1
    # else: # class ids match
    if True: # dont care about class id
      pred_bbox = det_info[0][1]
      gt_bbox = gt_det_info[1:5]
      pred_bbox[2] += pred_bbox[0]
      pred_bbox[3] += pred_bbox[1]

      # print('pred {} | gt {}'.format(pred_bbox, gt_bbox))

      iou = IOU(pred_bbox, gt_bbox)
      if iou >= iou_threshold:
        tp += 1
      else:
        fp += 1
  return tp, fp, fn


def extract_gt(m_gt_path):
  with open(m_gt_path) as fo:
    line = fo.readline()
    cnt = 1
    # print('gt line', line)
    while line:
      if cnt > 1:
        print('ERROR: more than 1 detection in frame. need to rewrite eval function')
        sys.exit()

      gt_det_info = line.strip().split(' ')
      gt_det_info = [float(gt_det_info[i]) for i in range(len(gt_det_info))]
      # print(gt_det_info)
      gt_det_info[1:5] = center2corner(gt_det_info[1:5])
      line = fo.readline()
      cnt += 1
  return gt_det_info

## Prediction Functions

In [0]:
def detect(image_path, Video = None, fig_size=(16,9), show_prediction=False, show_timing=True, show_blank=True, save_blank=True):
  det_info = []

  image = cv2.imread(image_path)
  (H, W) = image.shape[:2]
  # determine only the *output* layer names that we need from YOLO
  ln = net.getLayerNames()
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  # construct a blob from the input image and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes and
  # associated probabilities
  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (512, 288),
    swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)
  end = time.time()
  # show timing information on YOLO
  if show_timing:
    print("[INFO] YOLO took {:.6f} seconds".format(end - start))

  # initialize our lists of detected bounding boxes, confidences, and
  # class IDs, respectively
  boxes = []
  confidences = []
  classIDs = []

  # loop over each of the layer outputs
  for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
      # extract the class ID and confidence (i.e., probability) of
      # the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      if confidence > args_confidence:
        # scale the bounding box coordinates back relative to the
        # size of the image, keeping in mind that YOLO actually
        # returns the center (x, y)-coordinates of the bounding
        # box followed by the boxes' width and height
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")
        # use the center (x, y)-coordinates to derive the top and
        # and left corner of the bounding box
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))
        # update our list of bounding box coordinates, confidences,
        # and class IDs
        boxes.append([x, y, int(width), int(height)])
        confidences.append(float(confidence))
        classIDs.append(classID)
    
  # apply non-maxima suppression to suppress weak, overlapping bounding
  # boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, args_confidence, args_threshold)

  best_confidence = 0
  # ensure at least one detection exists
  if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
      # extract the bounding box coordinates
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])
      # draw a bounding box rectangle and label on the image
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(image, (x, y), (x + w, y + h), color, 6)
      text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
      cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        2.5, color, 6)
      
      if confidences[i] > best_confidence:
        det_info = [classIDs[i], boxes[i]], confidences[i]
        best_confidence = confidences[i]
  
  #save frames to videos  
  if Video is not None and save_blank:
    Video.write(image)


  # show the output image
  if show_prediction and (len(det_info) != 0 or show_blank):
    plt.figure(figsize = fig_size) #change figure size here. native aspect ratio is 16:9
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

  return det_info

def detect_track_mosse(vid_path, Video = None, N=5, show_prediction=True, show_timing=True, show_blank=False):
  output_bbox = False
  class Args:
    lr = 0.00125
    sigma = 0.1
    num_pretrain = 64
    rotate = True
    record = False

  args = Args()
  bounding_box_list = []


  frame = 0
  frame_temp = 0
  det_info_temp = []
  while frame < 30:
    # print('frame={}, frame_temp={}'.format(frame, frame_temp))
    for root, dirs, files in os.walk(vid_path):
      img_list = filter_and_sort(files)
      
      # repeat = False
      if frame_temp%N == 0:
        m_img_path = os.path.join(root, img_list[frame])
      

        det_info = detect(m_img_path,Video,fig_size=(15,4), show_prediction=show_prediction, show_timing=show_timing, show_blank=show_blank)
       

        if (len(det_info) == 0 and len(det_info_temp) != 0):
          # print('No detection, using prev det as template for tracking for cur frame')
          prev_det_bbox = det_info_temp[0][1]
          x = prev_det_bbox[0]
          y = prev_det_bbox[1]
          width = prev_det_bbox[2]
          height = prev_det_bbox[3]
          tracker = mosse(args, args_vid_path)

          # sometimes has fft size errors
          try:
            tracker.start_tracking((x, y, width, height), frame, frame+1, bounding_box_list, False, Video, show_prediction=show_prediction,show_timing=show_timing)
            bbox = bounding_box_list[-1]
          except:
            bounding_box_list.append([])
          
          # repeat = True
        elif len(det_info) != 0:
          # print('Detected')
          det_info_temp = det_info
          bbox = det_info[0][1]
          bounding_box_list.append(bbox)

          x = bbox[0]
          y = bbox[1]
          width = bbox[2]
          height = bbox[3]

        else:
          # print('No detections found so far')
          bounding_box_list.append([])
          frame_temp -= 1
        
        frame += 1
        frame_temp += 1
      else:
        tracker = mosse(args, args_vid_path)
        try:
          # print('2a len(bounding_box_list)={}'.format(len(bounding_box_list)))
          tracker.start_tracking((x, y, width, height), frame, frame+N-1, bounding_box_list, output_bbox,Video, show_prediction=show_prediction,show_timing=show_timing)
          frame += (N-1)
          frame_temp += N-1 
        except:
          # print('2b len(bounding_box_list)={}'.format(len(bounding_box_list)))
          bounding_box_list.append([])
          frame += 1
          frame_temp += 1


  return bounding_box_list

def detect_track_siamrpn(model_path, vid_path, Video=None, N=5, show_prediction=True, show_timing=True, show_blank=False):
  ### assume that siamrpn always detects an object

  tracker = TrackerSiamRPN(net_path=model_path)
  bounding_box_list = []
  template = []
  template_img = []
  frame = 0
  repeat = False

  for root, dirs, files in os.walk(vid_path):
    img_list = filter_and_sort(files)
    img_path_list = [os.path.join(root, img_list[i]) for i in range(len(img_list))]
    # print(img_path_list)
    while frame < len(img_path_list):
      # print('frame ',frame+1)
      m_img_path = img_path_list[frame]
      if frame%N == 0 or repeat: # do detection
        repeat = False
        det_info = detect(m_img_path,Video, fig_size=(15,4), show_prediction=show_prediction, show_timing=show_timing, show_blank=show_blank, save_blank=False)
        if len(det_info) != 0: # update template
          template = det_info[0][1]
          template_img_path = [m_img_path]
          # print(template)
          bounding_box_list.append(template)
        else: # did not detect any object, then do tracking with template
          if len(template) != 0: # if we have a valid template
            
            img_files = template_img_path + [img_path_list[frame]]
            boxes, times = tracker.track(img_files, template, visualize=show_prediction, Video = Video)

            for idx, mbox in enumerate(boxes):
              if idx > 0:
                bounding_box_list.append([mbox[0], mbox[1], mbox[2], mbox[3]])
          else: # did not find any detections, and no template
            bounding_box_list.append([])
        frame += 1
      else: # do tracking for N-1 frames
        end_frame = frame+N-1
        if end_frame > len(img_path_list):
          end_frame = len(img_path_list)

        # print(img_path_list[frame:end_frame])
        if len(template) != 0: # if we have valid template
          img_files = template_img_path + img_path_list[frame:end_frame]
          boxes, times = tracker.track(img_files, template, visualize=show_prediction, Video = Video)
          for idx, mbox in enumerate(boxes):
            if idx > 0:
              bounding_box_list.append([mbox[0], mbox[1], mbox[2], mbox[3]])
          frame += N-1 
        else: # no template stored, then cannot do tracking
          repeat = True
  if len(bounding_box_list) != len(img_path_list):
    print('ERROR: only recorded {} bounding boxes'.format(len(bounding_box_list)))
    sys.exit()
  return bounding_box_list

        

## Evaluation Functions


In [0]:
def evaluate_model(path, print_every=10, print_vid_metrics=False):
  TP = 0
  FP = 0
  FN = 0

  num_vids = 0
  total_vids = 0

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))
  start = time.time()

  for root1, dirs, _ in os.walk(path):
    total_vids = len(dirs)
    for mdir in dirs:
      vid_tp = 0
      vid_fp = 0
      vid_fn = 0
      for root2, _, images in os.walk(os.path.join(root1, mdir)):
        img_list = filter_and_sort(images)
        for m_img in img_list:
          if m_img[-1].isdigit():
            continue
          m_img_path = os.path.join(root2, m_img)

          # Run detection on current image
          det_info = detect(m_img_path, show_prediction=False, show_timing=False)

          # compare boxes with GT
          m_gt_path = os.path.join(gt_dir, m_img[0:-4]+'.txt')
          gt_det_info = extract_gt(m_gt_path)

          tp, fp, fn = eval_image(det_info, gt_det_info)

          vid_tp += tp
          vid_fp += fp
          vid_fn += fn

      TP += vid_tp
      FP += vid_fp
      FN += vid_fn

      if vid_tp == 0:
        vid_precision = 0
        vid_recall = 0
      else:
        vid_precision = vid_tp/(vid_tp + vid_fp)
        vid_recall = vid_tp/(vid_tp + vid_fn)

      if print_vid_metrics:
        print('[INFO] Video {} has precision = {} and recall = {}'.format(mdir, vid_precision, vid_recall))

      num_vids += 1
      if num_vids % print_every == 0:
        print('[INFO] Processed ', num_vids, ' videos out of ', total_vids, 'videos | minutes elapsed from start = ', int((time.time()-start)/60),'\n')

  precision = TP/(TP+FP)
  recall = TP/(TP+FN)

  print('[INFO] Precision = ', precision, ' | Recall = ', recall, ' at Confidence = ', args_confidence)

  end = time.time()
  print("[INFO] EVAL took {} minutes".format( int((end - start)/60)) )

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

  return precision, recall


def evaluate_model_mosse(path, N=5, print_every=10, print_vid_metrics=False):
  TP = 0
  FP = 0
  FN = 0

  num_vids = 0
  total_vids = 0

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))
  start = time.time()

  for root1, dirs, _ in os.walk(path):
    total_vids = len(dirs)
    for mdir in dirs:

      if mdir == 'track0095':
        print('skip track0095')
        continue

      vid_tp = 0
      vid_fp = 0
      vid_fn = 0
      full_vid_path = os.path.join(root1, mdir)
      bboxes = detect_track_mosse(full_vid_path, N=N, show_prediction=False, show_timing=False)
      for root2, _, images in os.walk(os.path.join(root1, mdir)):
        img_list = filter_and_sort(images)
        for frame, m_img in enumerate(img_list):
          if m_img[-1].isdigit():
            continue
          m_img_path = os.path.join(root2, m_img)
          m_gt_path = os.path.join(gt_dir, m_img[0:-4]+'.txt')
          gt_det_info = extract_gt(m_gt_path)
          
          det_info = bboxes[frame]
          if len(det_info) != 0:
            det_info = [[0, det_info]]
          tp, fp, fn = eval_image(det_info, gt_det_info)
          vid_tp += tp
          vid_fp += fp
          vid_fn += fn

      TP += vid_tp
      FP += vid_fp
      FN += vid_fn

      if vid_tp == 0:
        vid_precision = 0
        vid_recall = 0
      else:
        vid_precision = vid_tp/(vid_tp + vid_fp)
        vid_recall = vid_tp/(vid_tp + vid_fn)

      if print_vid_metrics:
        print('[INFO] Video {} has precision = {} and recall = {}'.format(mdir, vid_precision, vid_recall))

      num_vids += 1
      if num_vids % print_every == 0:
        print('[INFO] Processed ', num_vids, ' videos out of ', total_vids, 'videos | minutes elapsed from start = ', int((time.time()-start)/60),'\n')

  precision = TP/(TP+FP)
  recall = TP/(TP+FN)

  print('[INFO] Precision = ', precision, ' | Recall = ', recall, ' at Confidence = ', args_confidence)

  end = time.time()
  print("[INFO] EVAL took {} minutes".format( int((end - start)/60)) )

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

  return precision, recall

def evaluate_model_siamrpn(model_path,path, N=5, print_every=10, print_vid_metrics=False, Video = None):
  TP = 0
  FP = 0
  FN = 0

  num_vids = 0
  total_vids = 0

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))
  start = time.time()

  for root1, dirs, _ in os.walk(path):
    total_vids = len(dirs)
    for mdir in dirs:
      vid_tp = 0
      vid_fp = 0
      vid_fn = 0
      full_vid_path = os.path.join(root1, mdir)

      if mdir == 'track0095': #this video is missing frame [03], so just skip whole vid
        print('skip track0095')
        continue
      
      bboxes = detect_track_siamrpn(model_path, full_vid_path, N=N, show_prediction=False, show_timing=False)
  
      for root2, _, images in os.walk(os.path.join(root1, mdir)):
        img_list = filter_and_sort(images)
        for frame, m_img in enumerate(img_list):
          if m_img[-1].isdigit():
            continue
          m_img_path = os.path.join(root2, m_img)
          m_gt_path = os.path.join(gt_dir, m_img[0:-4]+'.txt')
    
          gt_det_info = extract_gt(m_gt_path)
          

          det_info = bboxes[frame]

          if len(det_info) != 0:
            det_info = [[0, det_info]]
          tp, fp, fn = eval_image(det_info, gt_det_info)
          vid_tp += tp
          vid_fp += fp
          vid_fn += fn

      TP += vid_tp
      FP += vid_fp
      FN += vid_fn

      if vid_tp == 0:
        vid_precision = 0
        vid_recall = 0
      else:
        vid_precision = vid_tp/(vid_tp + vid_fp)
        vid_recall = vid_tp/(vid_tp + vid_fn)

      if print_vid_metrics:
        print('[INFO] Video {} has precision = {} and recall = {}'.format(mdir, vid_precision, vid_recall))

      num_vids += 1
      if num_vids % print_every == 0:
        print('[INFO] Processed ', num_vids, ' videos out of ', total_vids, 'videos | minutes elapsed from start = ', int((time.time()-start)/60),'\n')

  if TP == 0:
    precision = 0
    recall = 0
  else:
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

  print('[INFO] Precision = ', precision, ' | Recall = ', recall, ' at Confidence = ', args_confidence)

  end = time.time()
  print("[INFO] EVAL took {} minutes".format( int((end - start)/60)) )

  utc_dt = datetime.now(timezone.utc)
  PST = pytz.timezone('US/Pacific')
  print("\nPacific time {}\n".format(utc_dt.astimezone(PST).isoformat()))

  return precision, recall

# Detection on single image

In [0]:
detect(args_img_path, fig_size=(15,5), show_prediction=True)

# Detection on single video

In [0]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')

out = cv2.VideoWriter('/content/drive/My Drive/TraCAR/track139_detonly.mp4',fourcc, 5, (1920, 1080))

frame = 0
for root, dirs, files in os.walk(args_vid_path):
  img_list = filter_and_sort(files)

  for m_img in img_list:

    m_img_path = os.path.join(root, m_img)

    det_info = detect(m_img_path, out, fig_size=(15,4), show_prediction=True)
    # print(det_info)

    print('frame ', frame)
    
    #save each frame into some path

    frame += 1 
out.release()


# detection + MOSSE
github link here

In [0]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('/content/drive/My Drive/TraCAR/track139_det+mosse_N=5.mp4',fourcc, 5, (1920, 1080))
boxes = detect_track_mosse(args_vid_path,out,N=5)
out.release()

#detection + siamrpn
https://github.com/huanglianghua/siamrpn-pytorch

In [0]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('/content/drive/My Drive/TraCAR/track139_det+siamrpn_N=5.mp4',fourcc, 5, (1920, 1080))
# show_prediction needs to be True to save video result
boxes = detect_track_siamrpn(siamrpn_path, args_vid_path, out, N=5, show_prediction=True) 
out.release()

#Evaluate detection only

In [0]:
precision, recall = evaluate_model(train_path, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model(test_path, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model(val_path, print_every=5, print_vid_metrics=True)

# Evaluate Detection + MOSSE

In [0]:
precision, recall = evaluate_model_mosse(train_path, N=10, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model_mosse(test_path, N=10, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model_mosse(val_path, N=10, print_every=5, print_vid_metrics=True)

# Evaluate Detection+SiamRPN

In [0]:
precision, recall = evaluate_model_siamrpn(siamrpn_path, train_path, N=10, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model_siamrpn(siamrpn_path, test_path, N=10, print_every=10, print_vid_metrics=True)

In [0]:
precision, recall = evaluate_model_siamrpn(siamrpn_path, val_path, N=10, print_every=5, print_vid_metrics=True)